# Inverse Sensor Model

Summary of notations for the sonar rangefinder inverse sensor model:

- $m_{i}$: Map at instant i or current cell that is being processed
- $x_{i},y_{i}$: Center of mass of the current cell mi
- $r$: Range of the center of mass computed with respect to robot pose and center of mass
- $k$: The sonar rangefinder cone that best aligns with the cell being considered computed with respect to the robot pose ($x$,$y$,$\theta$), center of mass ($x_i$,$y_i$), and sensor angle.
- $\beta$: Opening angle of the conical region formed out of the measurement beams.
- $\alpha$: Width of obstacles which is almost equal to the size of a cell. Please not that alpha is not the width of the conical region as the video mention but instead it's the width of a cell.

![](images/screen-shot-2018-02-22-at-4.57.09-pm.png)

In this quiz, you'll code the inverseSensorModel() function which has two separate tasks:

1. Compute $r$ and $\phi$
2. Evaluate the three different cases of the algorithm

In [1]:
#include <iostream>
#include <fstream>
#include <math.h>
#include <vector>
using namespace std;

In [2]:
// Sensor characteristic: Min and Max ranges of the beams
double Zmax = 5000, Zmin = 170;

// Defining free cells(lfree), occupied cells(locc), unknown cells(l0) log odds values
double l0 = 0, locc = 0.4, lfree = -0.4;

// Grid dimensions
double gridWidth = 100, gridHeight = 100;

// Map dimensions
double mapWidth = 30000, mapHeight = 15000;

// Robot size with respect to the map 
double robotXOffset = mapWidth / 5, robotYOffset = mapHeight / 3;

// Defining an l vector to store the log odds values of each cell
vector< vector<double> > l(mapWidth/gridWidth, vector<double>(mapHeight/gridHeight));

In [3]:
double inverseSensorModel(double x, double y, double theta, double xi, double yi, double sensorData[])
{
    //******************Code the Inverse Sensor Model Algorithm**********************//
    // Defining Sensor Characteristics
    double Zk, thetaK, sensorTheta;
    double minDelta = -1;
    double alpha = 200, beta = 20;

    //******************Compute r and phi**********************//
    double r = sqrt(pow(xi - x, 2) + pow(yi - y, 2));
    double phi = atan2(yi - y, xi - x) - theta;

    //Scaling Measurement to [-90 -37.5 -22.5 -7.5 7.5 22.5 37.5 90]
    for (int i = 0; i < 8; i++) {
        if (i == 0) {
            sensorTheta = -90 * (M_PI / 180);
        }
        else if (i == 1) {
            sensorTheta = -37.5 * (M_PI / 180);
        }
        else if (i == 6) {
            sensorTheta = 37.5 * (M_PI / 180);
        }
        else if (i == 7) {
            sensorTheta = 90 * (M_PI / 180);
        }
        else {
            sensorTheta = (-37.5 + (i - 1) * 15) * (M_PI / 180);
        }

        if (fabs(phi - sensorTheta) < minDelta || minDelta == -1) {
            Zk = sensorData[i];
            thetaK = sensorTheta;
            minDelta = fabs(phi - sensorTheta);
        }
    }

    //******************Evaluate the three cases**********************//
    if (r > min((double)Zmax, Zk + alpha / 2) || fabs(phi - thetaK) > beta / 2 || Zk > Zmax || Zk < Zmin) {
        return l0;
    }
    else if (Zk < Zmax && fabs(r - Zk) < alpha / 2) {
        return locc;
    }
    else if (r <= Zk) {
        return lfree;
    }
    
    return 0;
}

In [4]:
void occupancyGridMapping(double Robotx, double Roboty, double Robottheta, double sensorData[])
{
    //******************Code the Occupancy Grid Mapping Algorithm**********************//
    for (int x = 0; x < mapWidth / gridWidth; x++) {
        for (int y = 0; y < mapHeight / gridHeight; y++) {
            double xi = x * gridWidth + gridWidth / 2 - robotXOffset;
            double yi = -(y * gridHeight + gridHeight / 2) + robotYOffset;
            if (sqrt(pow(xi - Robotx, 2) + pow(yi - Roboty, 2)) <= Zmax) {
                l[x][y] = l[x][y] + inverseSensorModel(Robotx, Roboty, Robottheta, xi, yi, sensorData) - l0;
            }
        }
    }
}

In [5]:
double timeStamp;
double measurementData[8];
double robotX, robotY, robotTheta;

FILE* posesFile = fopen("poses.txt", "r");
FILE* measurementFile = fopen("measurement.txt", "r");

In [6]:
// Scanning the files and retrieving measurement and poses at each timestamp
while (fscanf(posesFile, "%lf %lf %lf %lf", &timeStamp, &robotX, &robotY, &robotTheta) != EOF) {
    fscanf(measurementFile, "%lf", &timeStamp);
    for (int i = 0; i < 8; i++) {
        fscanf(measurementFile, "%lf", &measurementData[i]);
    }
    occupancyGridMapping(robotX, robotY, (robotTheta / 10) * (M_PI / 180), measurementData);
}

In [7]:
// Displaying the map
for (int x = 0; x < mapWidth / gridWidth; x++) {
    for (int y = 0; y < mapHeight / gridHeight; y++) {
        cout << l[x][y] << " ";
    }
}

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [8]:
fclose(posesFile);
fclose(measurementFile);

We're now exporting the map as a PGM file; note that the image is rotated by 90 degrees due to the way the array is created.

In [9]:
ofstream map;
map.open("map.pgm");
map << "P2" << std::endl;
map << static_cast<uint32_t>(mapHeight / gridHeight) << " " 
    << static_cast<uint32_t>(mapWidth / gridWidth) << std::endl;
map << "255" << std::endl;

for (int x = 0; x < mapWidth / gridWidth; x++) {
    for (int y = 0; y < mapHeight / gridHeight; y++) {
        const auto p = 1 - std::exp(l[x][y]);
        const auto value = std::to_string(static_cast<uint8_t>(p * 255));
        map << value << " ";
    }
    map << std::endl;
}
map.close()

In [10]:
!convert map.pgm map.png

![](map.png)